In [1]:
import pandas as pd
from nilearn import image, plotting, masking
from scipy import ndimage 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/gilles/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ref = image.load_img('/data/odc/derivatives/spynoza/sub-tk/ses-odc2/func/sub-tk_ses-odc2_task-checkerboard_acq-07_run-02_bold_reference.nii.gz')
zmap = image.load_img('/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_left_over_right_zmap.nii.gz')

layers_left = image.load_img('/data/odc/derivatives/nighres/layers/sub-tk_left_cruise_gwb_layering_depth.nii.gz')
layers_right = image.load_img('/data/odc/derivatives/nighres/layers/sub-tk_right_cruise_gwb_layering_depth.nii.gz')
layers = image.math_img('layers_l + layers_r', layers_l=layers_left, layers_r=layers_right)

layers_bold = image.resample_to_img(layers, zmap, interpolation='nearest')
layers_bold.to_filename('/data/odc/zooi/depth_bold.nii.gz')

image.resample_to_img('/data/odc/zooi/V1.rh.nii.gz', zmap, 'nearest').to_filename('/data/odc/zooi/V1.rh.bold.nii.gz')
image.resample_to_img('/data/odc/zooi/V1.lh.nii.gz', zmap, 'nearest').to_filename('/data/odc/zooi/V1.lh.bold.nii.gz')

v1 = image.math_img('v1_lh + v1_rh', v1_lh='/data/odc/zooi/V1.lh.bold.nii.gz', v1_rh='/data/odc/zooi/V1.rh.bold.nii.gz')

v1 = image.new_img_like(zmap, ndimage.binary_dilation(v1.get_data(), iterations=2))
mask = image.math_img('(v1 > 0) & (depth > 0)', v1=v1, depth=layers_bold, zmap=zmap)

In [3]:
import glob
import re
reg = re.compile('.*/sub-tk_ses-odc2_run-(?P<run>[0-9\.]+)_(?P<stimulation>[a-z]+)_psc.nii.gz')

right = glob.glob('/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-*0_right_psc.nii.gz')
left = glob.glob('/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-*0_left_psc.nii.gz')
psc = left + right

df = []
depth_map = masking.apply_mask(layers_bold, mask)
z = masking.apply_mask(zmap, mask)

for fn in psc:
    print(fn)
    d = reg.match(fn).groupdict()
    d['task'] = int(float(d['run'])) % 2
    d['psc'] = masking.apply_mask(fn, mask)
    d['depth'] = depth_map
    d['z'] = z
    d['run'] = int(float(d['run']))
    
    tmp = pd.DataFrame(d)
    tmp.index.set_names('voxel', inplace=True)
    tmp.reset_index(inplace=True)
    
    df.append(tmp)
    
df = pd.concat(df)

/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-1.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-10.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-2.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-3.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-4.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-5.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-6.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-7.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-8.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7/sub-tk/ses-odc2/sub-tk_ses-odc2_run-9.0_left_psc.nii.gz
/data/odc/derivatives/modelfitting/glm7

In [4]:
tmp = df.pivot_table(columns=['stimulation', 'voxel'], index='run', values='psc')

contrast = tmp['left'] - tmp['right']

In [7]:
import scipy as sp

In [8]:
sp.stats.pearsonr(contrast.loc[:5].mean(), contrast.loc[5:].mean())

(0.4537281116955115, 0.0)

In [16]:
np.percentile(df['z'], 95)

2.1361276931476056